In [20]:
#!/usr/bin/env python3

# v0.1:fixing the problem that the feature of exon/mRNA is before gene
# v0.2: Use CDS to decide which transcript is primary
# v0.3: Fix the bug that if the CDS_parent include '\r' , the lst_len will be 0

# GFF must have CDS feature
# GFF must have ID and Parent in column 9

import re
import fileinput
from collections import defaultdict

gene_dict = defaultdict(list)
tx_pos_dict = defaultdict(list)
CDS_dict = defaultdict(list)

for line in fileinput.input('/home/guisen/GCF_005281545.1_ASM528154v1_genomic.gff'):
    if line.startswith("#"):
        continue
    content = line.split("\t")
    if len(content) <= 8:
        continue
    #print(content)
    '''
    if content[2] == 'gene':
        gene_id = re.search(r'ID=(.*?)[;\n]', content[8]).group(1)
        #print(gene_id)
        if gene_id not in gene_dict:
            gene_dict[gene_id] = []
    '''
    if content[2] == "transcript" or content[2] == "mRNA":

        # use regual expression to extract the gene ID
        # match the pattern ID=xxxxx; or ID=xxxxx
        
        tx_id = re.search(r'ID=(.*?)[;\n]',content[8]).group(1)
        tx_parent = re.search(r'Parent=(.*?)[;\n]',content[8]).group(1)
        tx_parent = tx_parent.strip() # remove the 'r' and '\n'

        # if the parent of transcript is not in the gene_dict, create it rather than append
        if tx_parent in gene_dict:
            gene_dict[tx_parent].append(tx_id)
        else:
            gene_dict[tx_parent] = [tx_id]
        tx_pos_dict[tx_id] = [content[0],content[3], content[4], content[6] ]
    # GFF must have CDS feature
    if content[2] == 'CDS':
        width = int(content[4]) - int(content[3])
        CDS_parent = re.search(r'Parent=(.*?)[;\n]',content[8]).group(1)
        CDS_parent = CDS_parent.strip() # strip the '\r' and '\n'
        CDS_dict[CDS_parent].append(width)

gff_file = open('Eak.txt','a')
for gene, txs in gene_dict.items():
    tmp = 0
    for tx in txs:
        tx_len = sum(CDS_dict[tx])
        if tx_len > tmp:
            lst_tx = tx
            tmp = tx_len
    tx_chrom = tx_pos_dict[lst_tx][0]
    tx_start = tx_pos_dict[lst_tx][1]
    tx_end   = tx_pos_dict[lst_tx][2]
    tx_strand = tx_pos_dict[lst_tx][3]
   # print("{gene}\t{tx}\t{chrom}\t{start}\t{end}\t{strand}".format(gene=gene,tx=lst_tx,chrom=tx_chrom,start=tx_start,end=tx_end, strand=tx_strand))
    #gff_file.write(f"{gene}\t{lst_tx}\t{tx_chrom}\t{tx_start}\t{tx_end}\t{tx_strand}\n")
gff_file.close()

defaultdict(<class 'list'>, {})
defaultdict(<class 'list'>, {'gene-slc25a48': ['rna-XM_033617728.1'], 'gene-glra1': ['rna-XM_033620666.1', 'rna-XM_033620672.1', 'rna-XM_033620678.1', 'rna-XM_033620686.1'], 'gene-g3bp1': ['rna-XM_033629617.1'], 'gene-sparc': ['rna-XM_033625137.1'], 'gene-atox1': ['rna-XM_033620350.1'], 'gene-slc36a1': ['rna-XM_033621851.1', 'rna-XM_033621868.1', 'rna-XM_033621859.1'], 'gene-mtnr1al': ['rna-XM_033617664.1'], 'gene-fat2': ['rna-XM_033619531.1', 'rna-XM_033619522.1'], 'gene-lman2': ['rna-XM_033632889.1', 'rna-XM_033632881.1'], 'gene-LOC117258837': ['rna-XM_033630027.1', 'rna-XM_033630017.1'], 'gene-selenot2': ['rna-XM_033630040.1'], 'gene-LOC117258709': ['rna-XM_033629813.1'], 'gene-rnf4': ['rna-XM_033632698.1'], 'gene-sqstm1': ['rna-XM_033629700.1', 'rna-XM_033629709.1', 'rna-XM_033629718.1'], 'gene-mrnip': ['rna-XM_033629740.1', 'rna-XM_033629731.1'], 'gene-LOC117258627': ['rna-XM_033629678.1', 'rna-XM_033629687.1']})
